# 🦷 Q&A Generation for RAFT Dataset (ULTRA FAST + HIGH QUALITY)

This notebook generates high-quality question-answer pairs from dental text chunks using **Google Gemini 2.0 Flash**.

**Features:**
- ⚡ Ultra fast parallel processing (20 concurrent requests)
- 🎯 Quality filters to skip junk chunks
- 📚 8-10 clinical/educational questions per chunk
- 💾 Auto-resume from checkpoint

**Input:** `chunks.jsonl` from notebook 01
**Output:** `qa_pairs.jsonl` for RAFT formatting

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q google-generativeai tqdm nest_asyncio aiohttp

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paths
DATA_DIR = "/content/drive/MyDrive/RAFT_dental_data"
CHUNKS_FILE = f"{DATA_DIR}/chunks.jsonl"
OUTPUT_FILE = f"{DATA_DIR}/qa_pairs.jsonl"

import os
if os.path.exists(CHUNKS_FILE):
    print(f"✓ Found chunks file")
else:
    print(f"✗ chunks.jsonl not found. Run notebook 01 first!")

In [ ]:
# ===== CONFIGURATION =====

# 👇 Paste your GCP Gemini API key here
GEMINI_API_KEY = ""  # @param {type:"string"}

# ⚡ ULTRA FAST SETTINGS
MAX_CONCURRENT = 20         # 20 parallel requests
DELAY_BETWEEN_BATCHES = 0.1 # Minimal delay
BATCH_SIZE = 20             # 20 chunks per batch

# Processing settings
TIMEOUT_HOURS = 12
CHECKPOINT_EVERY = 200
MAX_RETRIES = 2

print("⚡ ULTRA FAST Configuration:")
print(f"  Concurrent requests: {MAX_CONCURRENT}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Questions per chunk: 8-10")
print(f"  Timeout: {TIMEOUT_HOURS} hours")

## 2. Initialize Gemini 2.0 Flash (FAST)

In [ ]:
import google.generativeai as genai
import json
import time
import re
import random
import asyncio
import aiohttp
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import threading

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

# ⚡ Use FAST model with async
model = genai.GenerativeModel(
    model_name='gemini-2.0-flash',
    generation_config={
        'temperature': 0.7,
        'top_p': 0.9,
        'max_output_tokens': 3000,
    }
)

# Test connection
print("Testing Gemini 2.0 Flash connection...")
try:
    test_response = model.generate_content("Say 'OK' only")
    print(f"✓ Connected! Response: {test_response.text.strip()}")
except Exception as e:
    print(f"✗ Connection failed: {e}")

## 3. Load Chunks

In [ ]:
# Load all chunks
chunks = []
with open(CHUNKS_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        chunks.append(json.loads(line))

print(f"✓ Loaded {len(chunks)} total chunks")

## 4. Quality Filter + Fast Q&A Generator

In [ ]:
# ===== QUALITY FILTER =====

def is_valid_chunk(chunk: dict) -> bool:
    """Check if chunk has enough educational dental content."""
    text = chunk.get('text', '').lower()
    
    # Skip title pages, ToC, index, etc.
    skip_patterns = [
        'table of contents', 'contents', 'copyright', 'all rights reserved',
        'isbn', 'printed in', 'library of congress', 'publisher',
        'preface', 'acknowledgment', 'dedication', 'about the author',
        'index', 'bibliography', 'further reading', 'contributors',
        'first published', 'reprinted', 'edited by'
    ]
    
    for pattern in skip_patterns:
        if pattern in text[:500]:
            return False
    
    # Must have minimum content
    if len(text) < 300:
        return False
    
    # Should have dental/medical terms
    dental_terms = [
        'tooth', 'teeth', 'dental', 'oral', 'pulp', 'enamel', 'dentin',
        'gingiva', 'periodon', 'endo', 'crown', 'root', 'cavity', 'caries',
        'restoration', 'implant', 'occlus', 'mandib', 'maxill', 'bone',
        'tissue', 'nerve', 'treatment', 'patient', 'clinical', 'diagnosis',
        'procedure', 'symptom', 'disease', 'infect', 'inflam', 'pain',
        'surgery', 'anesthes', 'extract', 'fill', 'canal', 'apex',
        'mucosa', 'saliva', 'jaw', 'bite', 'chew', 'molar', 'incisor',
        'premolar', 'canine', 'amalgam', 'composite', 'cement', 'x-ray',
        'radiograph', 'bacteria', 'plaque', 'calculus', 'bleeding'
    ]
    
    term_count = sum(1 for term in dental_terms if term in text)
    return term_count >= 3


# Pre-filter chunks
valid_chunks = [c for c in chunks if is_valid_chunk(c)]
print(f"✓ Valid chunks: {len(valid_chunks)} / {len(chunks)} ({100*len(valid_chunks)//len(chunks)}%)")
print(f"  Filtered out {len(chunks) - len(valid_chunks)} low-quality chunks")

In [ ]:
# ===== DIRECT API CALLS WITH AIOHTTP (GUARANTEED PARALLEL) =====

import aiohttp

SYSTEM_PROMPT = """You are a dental educator creating exam questions. Generate 8-10 high-quality questions from the text.

RULES:
- Generate exactly 8-10 questions
- ONLY clinical/educational dental content
- NO questions about book titles, authors, editions, page numbers
- Answers must cite text using ##begin_quote##...##end_quote##
- Mix of question types: clinical scenarios, conceptual, procedural, definitions
- Mix of difficulty: easy, medium, hard
- Return [] if no educational content

Return JSON only:
[{"question":"...","answer":"Based on the text, ##begin_quote##...##end_quote##. This means...","type":"clinical","difficulty":"medium"}]"""

API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GEMINI_API_KEY}"


async def generate_qa_direct(session: aiohttp.ClientSession, chunk: dict, semaphore: asyncio.Semaphore) -> list:
    """Generate Q&A using direct API call - TRULY PARALLEL."""
    async with semaphore:
        prompt = f"""{SYSTEM_PROMPT}

Text ({chunk.get('category', 'Dental')}):
{chunk['text'][:3000]}

Generate 8-10 questions. JSON:"""

        payload = {
            "contents": [{"parts": [{"text": prompt}]}],
            "generationConfig": {
                "temperature": 0.7,
                "topP": 0.9,
                "maxOutputTokens": 3000
            }
        }

        for attempt in range(MAX_RETRIES):
            try:
                async with session.post(API_URL, json=payload, timeout=aiohttp.ClientTimeout(total=60)) as resp:
                    if resp.status == 429:
                        await asyncio.sleep(2)
                        continue
                    
                    data = await resp.json()
                    
                    if 'candidates' not in data:
                        return []
                    
                    text = data['candidates'][0]['content']['parts'][0]['text'].strip()

                    # Clean markdown
                    text = re.sub(r'^```json\s*', '', text)
                    text = re.sub(r'^```\s*', '', text)
                    text = re.sub(r'\s*```$', '', text)

                    qa_pairs = json.loads(text)

                    # Filter and add metadata
                    valid = []
                    for qa in qa_pairs:
                        q = qa.get('question', '').lower()
                        if any(s in q for s in ['title', 'author', 'edition', 'publisher', 'copyright', 'page']):
                            continue
                        if 'question' in qa and 'answer' in qa and len(qa['answer']) > 30:
                            qa['chunk_id'] = chunk.get('chunk_id', '')
                            qa['source'] = chunk.get('source', '')
                            qa['category'] = chunk.get('category', '')
                            qa['page_number'] = chunk.get('page_number', 0)
                            qa['context_text'] = chunk['text']
                            valid.append(qa)

                    return valid

            except Exception as e:
                if attempt < MAX_RETRIES - 1:
                    await asyncio.sleep(1)
                continue

        return []


print("✓ Direct API Q&A generator ready (GUARANTEED parallel with aiohttp)")

## 5. Test Generation

In [ ]:
# Test with a good chunk using direct API
import nest_asyncio
nest_asyncio.apply()

test_chunk = valid_chunks[100]
print(f"Testing: {test_chunk['source'][:60]}...")
print(f"Category: {test_chunk['category']}")
print(f"Text preview: {test_chunk['text'][:200]}...\n")

async def test_generation():
    semaphore = asyncio.Semaphore(1)
    connector = aiohttp.TCPConnector(limit=1)
    async with aiohttp.ClientSession(connector=connector) as session:
        return await generate_qa_direct(session, test_chunk, semaphore)

test_qa = asyncio.run(test_generation())
print(f"Generated {len(test_qa)} Q&A pairs:\n")

for i, qa in enumerate(test_qa[:5], 1):
    print(f"Q{i}: {qa['question']}")
    print(f"A{i}: {qa['answer'][:120]}...")
    print(f"Type: {qa.get('type')}, Difficulty: {qa.get('difficulty')}\n")

if len(test_qa) > 5:
    print(f"... and {len(test_qa) - 5} more questions")

## 6. Check Existing Progress

In [ ]:
# Check for existing Q&A pairs (for resume)
existing_qa = []
processed_chunk_ids = set()

if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                qa = json.loads(line)
                existing_qa.append(qa)
                processed_chunk_ids.add(qa.get('chunk_id', ''))
            except:
                continue
    print(f"✓ Found {len(existing_qa)} existing Q&A pairs")
    print(f"✓ {len(processed_chunk_ids)} chunks already processed")
else:
    print("No existing progress. Starting fresh.")

# Filter unprocessed chunks
chunks_to_process = [c for c in valid_chunks if c.get('chunk_id', '') not in processed_chunk_ids]
print(f"\n📋 Chunks to process: {len(chunks_to_process)}")
print(f"📝 Expected Q&A pairs: ~{len(chunks_to_process) * 9} (9 per chunk)")

# Estimate time (with fast settings)
est_time_min = len(chunks_to_process) / (MAX_CONCURRENT * 3) / 60  # ~3 per second per worker
print(f"⏱️ Estimated time: ~{est_time_min:.0f} minutes (~{est_time_min/60:.1f} hours)")

## 7. Generate All Q&A Pairs (ULTRA FAST)

In [ ]:
# ===== TRUE PARALLEL GENERATION WITH AIOHTTP =====

import nest_asyncio
nest_asyncio.apply()

# Increase concurrency - Gemini allows high rate
MAX_CONCURRENT = 50  # 50 parallel requests!

START_TIME = time.time()
END_TIME = START_TIME + (TIMEOUT_HOURS * 3600)

print("="*60)
print("🚀 STARTING TRUE PARALLEL Q&A GENERATION (aiohttp)")
print("="*60)
print(f"📅 Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⚡ Concurrent requests: {MAX_CONCURRENT}")
print(f"📊 Chunks to process: {len(chunks_to_process)}")
print(f"❓ Questions per chunk: 8-10")
print(f"💾 Output: {OUTPUT_FILE}")
print("="*60)
print()

all_qa = existing_qa.copy()
total_new = 0
failed_chunks = 0

async def process_all_chunks_fast(chunks_list):
    """Process all chunks with GUARANTEED parallel aiohttp calls."""
    global total_new, failed_chunks
    
    semaphore = asyncio.Semaphore(MAX_CONCURRENT)
    connector = aiohttp.TCPConnector(limit=MAX_CONCURRENT, limit_per_host=MAX_CONCURRENT)
    
    async with aiohttp.ClientSession(connector=connector) as session:
        
        # Process in batches of 1000
        BATCH_SIZE = 1000
        
        with open(OUTPUT_FILE, 'a' if existing_qa else 'w', encoding='utf-8') as f:
            for batch_start in range(0, len(chunks_list), BATCH_SIZE):
                batch_end = min(batch_start + BATCH_SIZE, len(chunks_list))
                batch = chunks_list[batch_start:batch_end]
                
                print(f"\n📦 Batch {batch_start//BATCH_SIZE + 1}/{(len(chunks_list)-1)//BATCH_SIZE + 1} ({len(batch)} chunks)")
                
                # Create all tasks
                tasks = [generate_qa_direct(session, chunk, semaphore) for chunk in batch]
                
                # Run with progress bar
                batch_qa = 0
                batch_failed = 0
                
                with tqdm(total=len(tasks), desc="Generating") as pbar:
                    for coro in asyncio.as_completed(tasks):
                        try:
                            result = await coro
                            if result:
                                for qa in result:
                                    f.write(json.dumps(qa, ensure_ascii=False) + "\n")
                                    all_qa.append(qa)
                                    total_new += 1
                                    batch_qa += 1
                            else:
                                batch_failed += 1
                                failed_chunks += 1
                        except Exception as e:
                            batch_failed += 1
                            failed_chunks += 1
                        pbar.update(1)
                
                f.flush()
                elapsed = (time.time() - START_TIME) / 60
                rate = total_new / elapsed if elapsed > 0 else 0
                chunks_done = batch_start + len(batch)
                chunks_rate = chunks_done / elapsed if elapsed > 0 else 0
                eta = (len(chunks_list) - chunks_done) / chunks_rate if chunks_rate > 0 else 0
                
                print(f"✅ +{batch_qa} Q&A (failed: {batch_failed}) | Total: {total_new} | {rate:.0f} Q&A/min | ETA: {eta:.0f} min")
                
                if time.time() > END_TIME:
                    print("⏰ Timeout!")
                    break

# Run
asyncio.run(process_all_chunks_fast(chunks_to_process))

# Final stats
elapsed_total = (time.time() - START_TIME) / 60

print()
print("="*60)
print("✅ GENERATION COMPLETE")
print("="*60)
print(f"📝 Total Q&A pairs: {len(all_qa)}")
print(f"✅ New Q&A generated: {total_new}")
print(f"❌ Failed chunks: {failed_chunks}")
print(f"⏱️ Time: {elapsed_total:.1f} min ({elapsed_total/60:.2f} hrs)")
if elapsed_total > 0:
    print(f"⚡ Speed: {total_new/elapsed_total:.0f} Q&A/min | {len(all_qa)/(elapsed_total)*9:.0f} chunks/min")
print(f"📁 Saved to: {OUTPUT_FILE}")
print("="*60)

## 8. Analyze Results

In [ ]:
# Load and analyze Q&A pairs
qa_pairs = []
with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            qa_pairs.append(json.loads(line))
        except:
            continue

print(f"Total Q&A pairs: {len(qa_pairs)}")

# By category
print("\n📊 By Category:")
cat_counts = {}
for qa in qa_pairs:
    cat = qa.get('category', 'Unknown')
    cat_counts[cat] = cat_counts.get(cat, 0) + 1
for cat, count in sorted(cat_counts.items(), key=lambda x: -x[1])[:10]:
    print(f"  {cat}: {count}")

# By type
print("\n📊 By Question Type:")
type_counts = {}
for qa in qa_pairs:
    t = qa.get('type', 'unknown')
    type_counts[t] = type_counts.get(t, 0) + 1
for t, count in sorted(type_counts.items(), key=lambda x: -x[1]):
    print(f"  {t}: {count}")

# By difficulty
print("\n📊 By Difficulty:")
diff_counts = {}
for qa in qa_pairs:
    d = qa.get('difficulty', 'unknown')
    diff_counts[d] = diff_counts.get(d, 0) + 1
for d, count in sorted(diff_counts.items(), key=lambda x: -x[1]):
    print(f"  {d}: {count}")

# Citation rate
with_citations = sum(1 for qa in qa_pairs if '##begin_quote##' in qa.get('answer', ''))
print(f"\n✅ Citation rate: {100*with_citations/len(qa_pairs):.1f}%")

In [ ]:
# Sample Q&A pairs
import random

print("📝 Sample Q&A pairs:\n")
print("="*60)

for qa in random.sample(qa_pairs, min(5, len(qa_pairs))):
    print(f"📚 {qa.get('category', 'N/A')}")
    print(f"❓ {qa.get('question', 'N/A')}")
    ans = qa.get('answer', '')
    print(f"✅ {ans[:250]}{'...' if len(ans) > 250 else ''}")
    print(f"   Type: {qa.get('type')} | Difficulty: {qa.get('difficulty')}")
    print("="*60)

## 9. Summary

In [ ]:
file_size = os.path.getsize(OUTPUT_FILE) / (1024**2)

print("="*60)
print("🎉 Q&A GENERATION COMPLETE")
print("="*60)
print(f"📝 Total Q&A pairs: {len(qa_pairs)}")
print(f"📁 Output: {OUTPUT_FILE}")
print(f"💾 Size: {file_size:.1f} MB")
print(f"📊 Categories: {len(cat_counts)}")
print(f"✅ Citation rate: {100*with_citations/len(qa_pairs):.1f}%")
print("="*60)
print("\n👉 Next: Run 03_raft_dataset_creation.ipynb")
print("="*60)